In [1]:
# 1. Installa Conda su Colab
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...


In [1]:
# 2. Verifica Conda e Clona il Repo
import condacolab
condacolab.check()

# Clona il repository ufficiale (se non esiste già)
import os
if not os.path.exists("EgoVLP"):
    !git clone https://github.com/showlab/EgoVLP.git

%cd EgoVLP

# Installa le dipendenze richieste tramite Conda
# Nota: PyTorch è già stato parzialmente installato nel passaggio precedente, qui ci assicuriamo delle versioni
!conda install -y pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

# --- FIX PER L'ERRORE TOKENIZERS ---
# Rimuoviamo "==4.12.5" da transformers per lasciare che pip trovi una versione compatibile con Python 3.11
# Rimuoviamo anche il blocco su timm se dovesse dare problemi, ma proviamo a mantenerlo per ora.
print("Installazione dipendenze Python...")
!pip install timm==0.4.12 transformers decord pandas h5py ftfy regex tqdm opencv-python

# Patch per il path
import sys
if "/content/EgoVLP" not in sys.path:
    sys.path.append("/content/EgoVLP")

✨🍰✨ Everything looks OK!
Cloning into 'EgoVLP'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 268 (delta 85), reused 100 (delta 47), pack-reused 109 (from 1)
Receiving objects: 100% (268/268), 2.00 MiB | 5.85 MiB/s, done.
Resolving deltas: 100% (114/114), done.
/content/EgoVLP
Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=11.3
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       7_kmp_llvm           8 KB  conda-forge
    aom-3.6.1                  |       h59595ed_0         2.6 MB  conda-forge
    blas-2.116                 |              mkl       

# PARTE 1: VIDEO EXTRACTOR

In [2]:
# 3. Preparazione Dati e Download Pesi Automatico
from google.colab import drive
import os
import shutil
import gdown # Libreria per scaricare da Google Drive

drive.mount('/content/drive')

# --- CONFIGURAZIONE PERCORSI ---
# Dove si trova lo zip dei tuoi video su Drive (questo deve essere corretto!)
PATH_ZIP_VIDEO = "/content/drive/MyDrive/MistakeDetection/captain_cook_4d_gopro_resized_8.zip"

# Dove salvare lo zip finale delle features
OUTPUT_ZIP_PATH = "/content/drive/MyDrive/MistakeDetection/egovlp_8.zip"
# -------------------------------

# 1. Setup Cartelle
if not os.path.exists("pretrained"):
    os.makedirs("pretrained")
if not os.path.exists("data/videos"):
    os.makedirs("data/videos")
if not os.path.exists("data/features"):
    os.makedirs("data/features")

# 2. Download Automatico dei Pesi (EgoVLP_PT_BEST.pth)
weights_path = "pretrained/model.pth"
if not os.path.exists(weights_path):
    print("⬇️ Download dei pesi EgoVLP_PT_BEST in corso...")
    # ID del file ufficiale dal GitHub di EgoVLP
    file_id = '1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7'
    url = f'https://drive.google.com/uc?id={file_id}'

    try:
        gdown.download(url, weights_path, quiet=False)
        print("✅ Pesi scaricati correttamente in pretrained/model.pth")
    except Exception as e:
        print(f"❌ Errore nel download: {e}")
        print("Prova a scaricarli manualmente da: https://drive.google.com/file/d/1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7/view")
else:
    print("✅ Pesi già presenti.")

# 3. Estrazione Video
# Controlla se abbiamo già estratto per non rifarlo ogni volta
if len(os.listdir("data/videos")) < 5: # Se la cartella è quasi vuota
    if os.path.exists(PATH_ZIP_VIDEO):
        print(f"📂 Estrazione video da {PATH_ZIP_VIDEO}...")
        !unzip -q "{PATH_ZIP_VIDEO}" -d "data/videos"
        print("✅ Video estratti.")
    else:
        print(f"❌ ERRORE: Non trovo il file zip dei video in: {PATH_ZIP_VIDEO}")
        print("Verifica il percorso nel tuo Google Drive!")
else:
    print("✅ Video già presenti in locale.")

Mounted at /content/drive
⬇️ Download dei pesi EgoVLP_PT_BEST in corso...


Downloading...
From (original): https://drive.google.com/uc?id=1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7
From (redirected): https://drive.google.com/uc?id=1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7&confirm=t&uuid=d2a1d366-34f4-4635-803c-94491e59b63a
To: /content/EgoVLP/pretrained/model.pth
100%|██████████| 2.17G/2.17G [00:21<00:00, 102MB/s]


✅ Pesi scaricati correttamente in pretrained/model.pth
📂 Estrazione video da /content/drive/MyDrive/MistakeDetection/captain_cook_4d_gopro_resized_8.zip...
✅ Video estratti.


In [3]:
import os

# Crea la cartella se non esiste
if not os.path.exists("pretrained"):
    os.makedirs("pretrained")

# Scarica il file dei pesi ImageNet richiesto hardcoded nel codice
print("⬇️ Download backbone ViT (richiesto dal codice originale)...")
!wget -nc https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth -O pretrained/jx_vit_base_p16_224-80ecf9dd.pth
print("✅ Backbone scaricato.")

⬇️ Download backbone ViT (richiesto dal codice originale)...
--2025-12-20 21:18:39--  https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/huggingface/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth [following]
--2025-12-20 21:18:39--  https://github.com/huggingface/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/168799526/65360900-1a09-11eb-8b86-f0a014a6f156?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-20T22%3A02%3A27Z&rscd=attachment%3B+filename%3Djx_vit_base_p16_224-80

In [4]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: torchvision 0.20.1
Uninstalling torchvision-0.20.1:
  Successfully uninstalled torchvision-0.20.1
Found existing installation: torchaudio 2.5.1
Uninstalling torchaudio-2.5.1:
  Successfully uninstalled torchaudio-2.5.1
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 152.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 171.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [5]:
%%writefile extract_custom.py
import torch
import torch.nn as nn
import glob
import os
import numpy as np
import cv2
import argparse
import time
import queue
import threading
from concurrent.futures import ThreadPoolExecutor
from model.model import FrozenInTime
from utils.util import state_dict_data_parallel_fix

# --- CONFIGURAZIONE PARALLELISMO ---
# Numero di video da leggere contemporaneamente (dipende dai core CPU)
# Su Colab standard 2-3 è ottimale. Su Pro anche 4-6.
NUM_WORKERS = 3
QUEUE_SIZE_PER_WORKER = 2
GLOBAL_QUEUE_SIZE = NUM_WORKERS * QUEUE_SIZE_PER_WORKER

# Parametri Normalizzazione (B, T, C, H, W)
MEAN = torch.tensor([0.485, 0.456, 0.406]).view(1, 1, 3, 1, 1)
STD = torch.tensor([0.229, 0.224, 0.225]).view(1, 1, 3, 1, 1)

def load_model(ckpt_path, device):
    print(f"Costruzione modello FrozenInTime su {device}...")
    model = FrozenInTime(
        video_params={
            "model": "SpaceTimeTransformer",
            "arch_config": "base_patch16_224",
            "num_frames": 16,
            "pretrained": True,
            "time_init": "zeros"
        },
        text_params={"model": "bert-base-uncased", "pretrained": True, "input": "text"},
        projection_dim=256,
        load_checkpoint=None,
        projection='minimal',
        load_temporal_fix='zeros'
    )
    checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    else:
        state_dict = checkpoint
    new_state_dict = state_dict_data_parallel_fix(state_dict, model.state_dict())
    model.load_state_dict(new_state_dict, strict=False)
    model = model.to(device)
    model.eval()
    return model

def preprocess_on_gpu(batch_uint8):
    """Normalizza batch uint8 [B, T, H, W, C] -> float32 [B, T, C, H, W]"""
    batch = batch_uint8.float() / 255.0
    batch = batch.permute(0, 1, 4, 2, 3).contiguous()
    batch = (batch - MEAN) / STD
    return batch

def video_worker(vpath, clip_len, batch_size, global_queue):
    """
    Worker che processa un singolo video e invia i batch alla coda globale.
    Invia tuple: (vpath, batch_tensor, is_last_batch)
    """
    try:
        cap = cv2.VideoCapture(vpath)
        if not cap.isOpened():
            # Segnala fine con errore
            global_queue.put((vpath, None, True))
            return

        buffer = []
        current_clip = []

        # Pre-calcolo crop 224x224
        resize_dim = 256
        crop_dim = 224
        start_y = (resize_dim - crop_dim) // 2
        start_x = (resize_dim - crop_dim) // 2

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # CPU Preprocessing
            frame = cv2.resize(frame, (resize_dim, resize_dim), interpolation=cv2.INTER_LINEAR)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = frame[start_y:start_y+crop_dim, start_x:start_x+crop_dim]

            current_clip.append(frame)

            if len(current_clip) == clip_len:
                buffer.append(np.stack(current_clip))
                current_clip = []

                if len(buffer) >= batch_size:
                    batch_np = np.stack(buffer)
                    batch_tensor = torch.from_numpy(batch_np)
                    # Mette in coda (blocca se piena per non saturare RAM)
                    global_queue.put((vpath, batch_tensor, False))
                    buffer = []

        # Residui Clip
        if len(current_clip) > 0:
            last_frame = current_clip[-1]
            pad_needed = clip_len - len(current_clip)
            current_clip.extend([last_frame] * pad_needed)
            buffer.append(np.stack(current_clip))

        # Residui Batch
        if len(buffer) > 0:
            batch_np = np.stack(buffer)
            batch_tensor = torch.from_numpy(batch_np)
            global_queue.put((vpath, batch_tensor, False))

        cap.release()
        # Segnale di fine video
        global_queue.put((vpath, None, True))

    except Exception as e:
        print(f"Error reading {vpath}: {e}")
        global_queue.put((vpath, None, True))

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--video_dir', type=str, required=True)
    parser.add_argument('--output_dir', type=str, required=True)
    parser.add_argument('--weights', type=str, required=True)
    parser.add_argument('--batch_size', type=int, default=96)
    args = parser.parse_args()

    if torch.cuda.is_available():
        device = 'cuda'
        print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
        print(f"⚡ MULTI-VIDEO MODE: {NUM_WORKERS} Workers Paralleli | Batch {args.batch_size}")
    else:
        device = 'cpu'
        print("⚠️ CUDA non disponibile.")

    global MEAN, STD
    MEAN = MEAN.to(device)
    STD = STD.to(device)

    model = load_model(args.weights, device)

    # Lista video da processare
    all_videos = []
    for ext in ['*.mp4', '*.avi', '*.MOV']:
        all_videos.extend(glob.glob(os.path.join(args.video_dir, "**", ext), recursive=True))

    # Filtra quelli già fatti
    videos_to_process = []
    for vpath in all_videos:
        vname = os.path.basename(vpath)
        save_name = f"{vname}_1s_1s.npz"
        save_path = os.path.join(args.output_dir, save_name)
        if not os.path.exists(save_path):
            videos_to_process.append(vpath)

    print(f"Trovati {len(all_videos)} video totali. Da elaborare: {len(videos_to_process)}")

    if not videos_to_process:
        print("Tutti i video completati!")
        return

    # Strutture dati condivise
    global_queue = queue.Queue(maxsize=GLOBAL_QUEUE_SIZE)
    results_buffer = {} # {vpath: [list_of_features]}
    active_videos = set()
    completed_videos = 0
    start_times = {}

    # Executor per i worker
    executor = ThreadPoolExecutor(max_workers=NUM_WORKERS)

    # Funzione helper per avviare un worker
    video_iterator = iter(videos_to_process)

    def start_next_worker():
        try:
            vpath = next(video_iterator)
            active_videos.add(vpath)
            results_buffer[vpath] = []
            start_times[vpath] = time.time()
            executor.submit(video_worker, vpath, 16, args.batch_size, global_queue)
            print(f"▶️ Start: {os.path.basename(vpath)}")
            return True
        except StopIteration:
            return False

    # Avvia i primi N worker
    for _ in range(NUM_WORKERS):
        start_next_worker()

    # MAIN LOOP (GPU Consumer)
    while active_videos:
        try:
            # Preleva un batch dalla coda (da qualsiasi video)
            vpath, batch_cpu, is_last = global_queue.get(timeout=60) # Timeout per evitare deadlock

            if is_last:
                # Video finito: Salva e chiudi
                features = np.concatenate(results_buffer[vpath], axis=0)

                vname = os.path.basename(vpath)
                save_name = f"{vname}_1s_1s.npz"
                save_path = os.path.join(args.output_dir, save_name)
                np.savez_compressed(save_path, features=features.astype(np.float32))

                elapsed = time.time() - start_times[vpath]
                fps = (features.shape[0] * 16) / elapsed
                completed_videos += 1
                print(f"✅ Saved: {vname} ({elapsed:.1f}s - {fps:.0f} fps) | Progress: {completed_videos}/{len(videos_to_process)}")

                # Pulizia
                del results_buffer[vpath]
                del start_times[vpath]
                active_videos.remove(vpath)

                # Avvia prossimo video
                start_next_worker()
                global_queue.task_done()
                continue

            if batch_cpu is not None:
                # Inferenza GPU
                batch_gpu = batch_cpu.to(device, non_blocking=True)
                input_batch = preprocess_on_gpu(batch_gpu)

                with torch.amp.autocast('cuda'):
                    with torch.no_grad():
                        v_embed = model.compute_video(input_batch)

                # Salva risultato nel buffer specifico del video
                results_buffer[vpath].append(v_embed.cpu().numpy().astype(np.float16))

            global_queue.task_done()

        except queue.Empty:
            print("⚠️ Warning: GPU Queue empty for 60s. Workers might be stuck.")
            break
        except Exception as e:
            print(f"❌ Critical Error in Main Loop: {e}")
            break

    executor.shutdown()
    print("🎉 Tutti i video elaborati.")

if __name__ == "__main__":
    main()

Writing extract_custom.py


In [6]:
!pip install einops
!pip install humanize psutil
!pip install ffmpeg-python
!pip install av dominate scikit-learn tensorboardx

# Crea cartella output
!mkdir -p data/features

# Esegui lo script
# Assicurati che VIDEO_ROOT punti alla cartella dove ci sono effettivamente i file .mp4
!python extract_custom.py \
  --video_dir "data/videos/captain_cook_4d_gopro_resized_8" \
  --output_dir "data/features" \
  --weights "pretrained/model.pth" \
  --batch_size 64

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 154.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 63.5 MB/s eta 0:00:00


🚀 GPU: Tesla T4
⚡ MULTI-VIDEO MODE: 3 Workers Paralleli | Batch 64
Costruzione modello FrozenInTime su cuda...
config.json: 100% 570/570 [00:00<00:00, 3.95MB/s]
model.safetensors: 100% 440M/440M [00:03<00:00, 139MB/s]
/content/EgoVLP/model/model.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don'

In [7]:
print("Creazione archivio zip...")
!zip -r -q egovlp_features.zip data/features

print(f"Upload su Drive: {OUTPUT_ZIP_PATH}")
shutil.copy("egovlp_features.zip", OUTPUT_ZIP_PATH)
print("✅ Completato!")

Creazione archivio zip...
Upload su Drive: /content/drive/MyDrive/MistakeDetection/egovlp_8.zip
✅ Completato!


# PARTE 2: TEXT ENCODER

In [7]:
# @title 🚀 Setup Dipendenze Substep 3 (Versione Pulita)
import sys
import os
import traceback

print("🔄 Preparazione ambiente per Text Encoder...")

# 1. BINARI DI SISTEMA (Video/Audio)
#    Usiamo Mamba per installare le librerie C++ sottostanti (ffmpeg, av, decord)
print("🔧 Installazione librerie di sistema (av, decord, ffmpeg)...")
get_ipython().system('mamba install -y av decord ffmpeg -c conda-forge')

# 2. PACCHETTI PYTHON
#    Installiamo direttamente la lista di pacchetti necessari
print("🔧 Installazione pacchetti Python...")
get_ipython().system(f'{sys.executable} -m pip install av decord ftfy regex tqdm simplejson pandas transformers ffmpeg-python')

print("✅ Installazioni completate.")

# 3. CONFIGURAZIONE PATH
repo_path = "/content/EgoVLP"
if os.path.exists(repo_path):
    if repo_path not in sys.path:
        sys.path.append(repo_path)
    os.chdir(repo_path)
    print(f"📂 Directory di lavoro: {os.getcwd()}")
else:
    print(f"⚠️ Cartella {repo_path} non trovata. Assicurati di aver scaricato la repo!")

# 4. TEST IMPORTAZIONE MODELLO
print("\n⚙️ Test Finale Importazione...")
try:
    import av
    import decord
    from model.model import FrozenInTime
    print("\n🎉 TUTTO OK! Il modello è pronto.")
    print("   Puoi procedere con le celle 2 (Caricamento Pesi) e 3 (Generazione).")
except ImportError as e:
    print(f"\n❌ ERRORE: {e}")
    print("⚠️ Se l'errore riguarda 'av' o 'decord', riavvia la sessione (Runtime -> Restart Session) e riesegui questa cella.")

🔄 Preparazione ambiente per Text Encoder...
🔧 Installazione librerie di sistema (av, decord, ffmpeg)...

Looking for: ['av', 'decord', 'ffmpeg']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
Your pinning does not match what's currently installed. Please remove the pin and fix your installation
  Pin: python=3.12
  Currently installed: conda-forge/linux-64::python==3.11.11=h9e4cc4f_1_cpython
🔧 Installazione pacchetti Python...
✅ Installazioni completate.
📂 Directory di lavoro: /content/EgoVLP

⚙️ Test Finale Importazione...

🎉 TUTTO OK! Il modello è pronto.
   Puoi procedere con le celle 2 (Caricamento Pesi) e 3 (Generazione).


In [9]:
# @title ⬇️ Download Pesi Modello (Esegui prima di caricare il modello)
import os
import gdown
import urllib.request

# Crea cartella pesi
if not os.path.exists("pretrained"):
    os.makedirs("pretrained")

print("📦 Controllo file dei pesi...")

# --- 1. DOWNLOAD BACKBONE ViT (Quello che ha causato il tuo errore) ---
# Questo file è richiesto dalla classe SpaceTimeTransformer per inizializzarsi
vit_path = "pretrained/jx_vit_base_p16_224-80ecf9dd.pth"
vit_url = "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth"

if not os.path.exists(vit_path):
    print(f"⬇️ Scaricando backbone ViT (jx_vit_base_p16_224-80ecf9dd.pth)...")
    try:
        urllib.request.urlretrieve(vit_url, vit_path)
        print("✅ Backbone ViT scaricato.")
    except Exception as e:
        print(f"❌ Errore download ViT: {e}")
else:
    print("✅ Backbone ViT già presente.")

# --- 2. DOWNLOAD CHECKPOINT EGOVLP (model.pth) ---
model_path = "pretrained/model.pth"
if not os.path.exists(model_path):
    print(f"⬇️ Scaricando checkpoint EgoVLP (model.pth)...")
    # ID del file drive ufficiale EgoVLP
    file_id = '1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7'
    url = f'https://drive.google.com/uc?id={file_id}'

    try:
        gdown.download(url, model_path, quiet=False)
        print("✅ Checkpoint EgoVLP scaricato.")
    except Exception as e:
        print(f"❌ Errore download model.pth: {e}")
else:
    print("✅ Checkpoint EgoVLP già presente.")

print("\n🎉 Pesi pronti! Ora puoi eseguire la Cella 2.")

📦 Controllo file dei pesi...
⬇️ Scaricando backbone ViT (jx_vit_base_p16_224-80ecf9dd.pth)...
✅ Backbone ViT scaricato.
⬇️ Scaricando checkpoint EgoVLP (model.pth)...


Downloading...
From (original): https://drive.google.com/uc?id=1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7
From (redirected): https://drive.google.com/uc?id=1-cP3Gcg0NGDcMZalgJ_615BQdbFIbcj7&confirm=t&uuid=bc3f90b1-b2a9-4b0a-9f09-756bde454c9f
To: /content/EgoVLP/pretrained/model.pth
100%|██████████| 2.17G/2.17G [00:22<00:00, 95.2MB/s]

✅ Checkpoint EgoVLP scaricato.

🎉 Pesi pronti! Ora puoi eseguire la Cella 2.


In [12]:
# @title 2. Inizializzazione Modello (Fix LOCAL_RANK)
import os
import torch
from transformers import AutoTokenizer

# --- FIX CRITICO PER EGOVLP ---
# Simuliamo le variabili d'ambiente distribuite per evitare l'errore
os.environ["LOCAL_RANK"] = "0"
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12345"
# ------------------------------

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚙️ Setup su device: {device}")

# 1. Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# 2. Configurazione
config = {
    "video_params": {
        "model": "SpaceTimeTransformer",
        "arch_config": "base_patch16_224",
        "num_frames": 16,
        "pretrained": True,
        "time_init": "zeros",
        "vit_ckpt": "pretrained/jx_vit_base_p16_224-80ecf9dd.pth"
    },
    "text_params": {
        "model": "distilbert-base-uncased",
        "pretrained": True,
        "input": "text"
    },
    "projection": "minimal",
    "load_checkpoint": "pretrained/model.pth"
}

# 3. Caricamento Pesi
print("⚙️ Caricamento pesi modello...")
try:
    # Importiamo qui per essere sicuri che prenda le env vars settate sopra
    from model.model import FrozenInTime

    model = FrozenInTime(**config)
    model = model.to(device)
    model.eval()
    print("✅ SUCCESSO! Modello EgoVLP caricato correttamente.")
except Exception as e:
    print(f"❌ Errore caricamento: {e}")
    # Stampa dettagliata per debug
    import traceback
    traceback.print_exc()

⚙️ Setup su device: cpu
⚙️ Caricamento pesi modello...
######USING ATTENTION STYLE:  frozen-in-time
❌ Errore caricamento: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.


Traceback (most recent call last):
  File "/tmp/ipython-input-4118718561.py", line 46, in <cell line: 0>
    model = FrozenInTime(**config)
            ^^^^^^^^^^^^^^^^^^^^^^
  File "/content/EgoVLP/model/model.py", line 91, in __init__
    checkpoint = torch.load(load_checkpoint, map_location='cuda:{}'.format(local_rank))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 1521, in load
    return _load(
           ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 2122, in _load
    result = unpickler.load()
             ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/_weights_only_unpickler.py", line 535, in load
    self.append(self.persistent_load(pid))
                ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 2086, in persistent_load
    typed_storage

In [11]:
# @title 3. Generazione Text Embeddings (Esecuzione Finale)
import numpy as np
import json
import glob
from tqdm import tqdm

# --- CONFIGURAZIONE ---
INPUT_GRAPHS_DIR = "/content/drive/MyDrive/MistakeDetection/task_graphs" # <--- VERIFICA QUESTO PERCORSO
OUTPUT_TEXT_EMB_DIR = "/content/drive/MyDrive/MistakeDetection/text_embeddings"

# Funzione helper
def encode_descriptions(descriptions, model, tokenizer, device):
    tokens = tokenizer(descriptions, padding=True, truncation=True, max_length=32, return_tensors="pt").to(device)
    with torch.no_grad():
        text_embeddings = model.compute_text(tokens)
    return text_embeddings.cpu().numpy()

# Creazione cartella output
if not os.path.exists(OUTPUT_TEXT_EMB_DIR):
    os.makedirs(OUTPUT_TEXT_EMB_DIR)

# Trova file
json_files = glob.glob(os.path.join(INPUT_GRAPHS_DIR, "*.json"))
print(f"📂 Trovati {len(json_files)} file JSON in: {INPUT_GRAPHS_DIR}")

if len(json_files) == 0:
    print("⚠️ NESSUN FILE TROVATO! Controlla il percorso INPUT_GRAPHS_DIR.")
else:
    count = 0
    for json_path in tqdm(json_files, desc="Processando Ricette"):
        try:
            filename = os.path.basename(json_path)
            recipe_name = filename.replace(".json", "")
            save_path = os.path.join(OUTPUT_TEXT_EMB_DIR, f"{recipe_name}_text.npz")

            with open(json_path, 'r') as f:
                data = json.load(f)

            if "steps" not in data: continue

            steps_dict = data["steps"]
            # Ordina per ID numerico
            sorted_ids = sorted(steps_dict.keys(), key=lambda x: int(x) if x.isdigit() else x)

            step_ids = []
            descriptions = []

            for sid in sorted_ids:
                desc = steps_dict[sid]
                # Gestione formato stringa o dizionario
                desc_text = desc['description'] if isinstance(desc, dict) and 'description' in desc else str(desc)
                step_ids.append(sid)
                descriptions.append(desc_text.strip())

            if not descriptions: continue

            # Encoding
            emb_matrix = encode_descriptions(descriptions, model, tokenizer, device)

            # Salvataggio
            np.savez_compressed(save_path, features=emb_matrix, step_ids=step_ids, text=descriptions)
            count += 1

        except Exception as e:
            print(f"❌ Errore su {filename}: {e}")

    print(f"\n🎉 COMPLETATO! Generati {count} file .npz in {OUTPUT_TEXT_EMB_DIR}")

⚙️ Setup su device: cpu
⚙️ Caricamento pesi modello...
######USING ATTENTION STYLE:  frozen-in-time
❌ Errore caricamento: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.


Traceback (most recent call last):
  File "/tmp/ipython-input-4118718561.py", line 46, in <cell line: 0>
    model = FrozenInTime(**config)
            ^^^^^^^^^^^^^^^^^^^^^^
  File "/content/EgoVLP/model/model.py", line 91, in __init__
    checkpoint = torch.load(load_checkpoint, map_location='cuda:{}'.format(local_rank))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 1521, in load
    return _load(
           ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 2122, in _load
    result = unpickler.load()
             ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/_weights_only_unpickler.py", line 535, in load
    self.append(self.persistent_load(pid))
                ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/serialization.py", line 2086, in persistent_load
    typed_storage

In [ ]:
# @title 3. Generazione Text Embeddings
# --- CONFIGURAZIONE ---
# Cartella INPUT: Dove sono i tuoi file .json (es. blenderbananapancakes.json)
INPUT_GRAPHS_DIR = "/content/drive/MyDrive/MistakeDetection/task_graphs"

# Cartella OUTPUT: Dove salvare gli embedding testuali
OUTPUT_TEXT_EMB_DIR = "/content/drive/MyDrive/MistakeDetection/text_embeddings"
# ----------------------

# Funzione helper per l'encoding
def encode_descriptions(descriptions, model, tokenizer, device):
    # Tokenizzazione con padding/truncation
    tokens = tokenizer(
        descriptions,
        padding=True,
        truncation=True,
        max_length=32, # EgoVLP è addestrato su clip brevi, 32-64 token bastano
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        # model.compute_text restituisce l'embedding proiettato (dimensione 256)
        text_embeddings = model.compute_text(tokens)
    return text_embeddings.cpu().numpy()

# Creazione cartella output
os.makedirs(OUTPUT_TEXT_EMB_DIR, exist_ok=True)

# Trova tutti i file JSON
json_files = glob.glob(os.path.join(INPUT_GRAPHS_DIR, "*.json"))
print(f"📂 Trovati {len(json_files)} file JSON in: {INPUT_GRAPHS_DIR}")

if len(json_files) == 0:
    print("❌ ERRORE: Nessun file trovato. Controlla il percorso INPUT_GRAPHS_DIR!")
else:
    count = 0
    for json_path in tqdm(json_files, desc="Processando Ricette"):
        filename = os.path.basename(json_path)
        recipe_name = filename.replace(".json", "")
        save_path = os.path.join(OUTPUT_TEXT_EMB_DIR, f"{recipe_name}_text.npz")

        try:
            with open(json_path, 'r') as f:
                data = json.load(f)

            # Estrazione Steps dal JSON
            # Ci aspettiamo una struttura: { "steps": { "0": "desc...", "1": "desc..." } }
            if "steps" not in data:
                print(f"⏩ Salto {filename}: chiave 'steps' mancante.")
                continue

            steps_dict = data["steps"]

            # Ordiniamo gli step per ID (importante per mantenere la coerenza col grafo)
            sorted_ids = sorted(steps_dict.keys(), key=lambda x: int(x) if x.isdigit() else x)

            step_ids = []
            descriptions = []

            for sid in sorted_ids:
                desc = steps_dict[sid]
                # A volte la descrizione è dentro un oggetto, a volte è stringa diretta
                desc_text = desc['description'] if isinstance(desc, dict) and 'description' in desc else str(desc)

                # Rimuoviamo eventuali ID hardcodati nel testo (es. "1. Wash") per pulizia
                desc_text = desc_text.strip()

                step_ids.append(sid)
                descriptions.append(desc_text)

            if not descriptions:
                continue

            # Generazione Embedding (Text Encoder)
            emb_matrix = encode_descriptions(descriptions, model, tokenizer, device)

            # Salvataggio .npz
            # Salviamo tutto ciò che serve per il matching successivo
            np.savez_compressed(
                save_path,
                features=emb_matrix,    # Matrice (Numero_Step x 256)
                step_ids=step_ids,      # Lista ID (es. ['0', '1', '2'])
                text=descriptions       # Testo originale (per debug e visualizzazione)
            )
            count += 1

        except Exception as e:
            print(f"❌ Errore su {filename}: {e}")

    print(f"\n🎉 COMPLETATO!")
    print(f"   Generati {count} file di embedding testuali.")
    print(f"   Salvato in: {OUTPUT_TEXT_EMB_DIR}")